In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os

In [2]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32
directory="Datav3"
categories=["with_mask","wo_mask"]

In [3]:
print("[INFO] loading images...")

data = []
labels = []
for imagePath in categories:
    path=os.path.join(directory,imagePath)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image = load_img(img_path, target_size=(128, 128))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(imagePath)

[INFO] loading images...


In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)


aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(128, 128, 3)))
baseModel

In [6]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(48, 48, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(48, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [7]:
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


C:\Users\Rajath.DESKTOP-ELKQ6GV\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:367: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
print("[INFO] training head...")
H = model.fit(aug.flow(trainX, trainY, batch_size=BS),steps_per_epoch=len(trainX) // BS, validation_data=(testX, testY),
    validation_steps=len(testX) // BS,
    epochs=EPOCHS)

# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] training head...
Epoch 1/20
1680/1680 [==============================] - 142s 83ms/step - loss: 0.5654 - accuracy: 0.7156 - val_loss: 0.5054 - val_accuracy: 0.7515
Epoch 2/20
1680/1680 [==============================] - 121s 72ms/step - loss: 0.4672 - accuracy: 0.7845
Epoch 3/20
1680/1680 [==============================] - 115s 69ms/step - loss: 0.4462 - accuracy: 0.7972
Epoch 4/20
1680/1680 [==============================] - 115s 68ms/step - loss: 0.4349 - accuracy: 0.8010
Epoch 5/20
1680/1680 [==============================] - 104s 62ms/step - loss: 0.4246 - accuracy: 0.8089
Epoch 6/20
1680/1680 [==============================] - 104s 62ms/step - loss: 0.4229 - accuracy: 0.8072
Epoch 7/20
1680/1680 [==============================] - 104s 62ms/step - loss: 0.4200 - accuracy: 0.8097
Epoch 8/20
1680/1680 [==============================] - 104s 62ms/step - loss: 0.4144 - accuracy: 0.8109
Epoch 9/20
1680/1680 [==============================] - 104s 62ms/step - loss: 0.4084 - accura

In [9]:
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.78      0.82      0.80      6737
     wo_mask       0.81      0.77      0.79      6711

    accuracy                           0.80     13448
   macro avg       0.80      0.80      0.80     13448
weighted avg       0.80      0.80      0.80     13448

